# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
import sys
key = "C:\\Users\\Kelly\\Desktop\\KEYS"
sys.path.append(key)
from GoogleAPI import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('clean_city_data.csv')
city_df

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Country,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
0,0,5880054,Barrow,1.0,70.0,US,0.76,-26.72,-7.60,-7.60,1026.0,69.0,NaN,NaN,89.562846,-162.300527
1,1,1337619,Ugoofaaru,49.0,76.0,MV,82.08,82.69,82.08,82.08,1013.0,67.0,1013.0,1013.0,5.914250,62.148229
2,2,2108502,Honiara,40.0,340.0,SB,9.14,98.71,91.40,91.40,1003.0,75.0,NaN,NaN,-10.979518,158.747830
3,3,3372964,Lagoa,100.0,233.0,PT,61.09,54.27,61.09,61.09,1015.0,76.0,1015.0,1009.0,47.945889,-27.492200
4,4,3460967,Itaberá,36.0,146.0,BR,67.33,71.06,67.33,67.33,1012.0,92.0,1012.0,942.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,555,2037355,Fushun,0.0,250.0,CN,1.22,1.83,12.20,12.20,1035.0,39.0,NaN,NaN,43.670723,134.349061
556,556,1488903,Turukhansk,100.0,187.0,RU,15.27,-29.85,-15.27,-15.27,1026.0,88.0,1026.0,1020.0,-29.647027,16.789820
557,557,538061,Kuybyshevo,67.0,356.0,RU,30.74,20.86,30.74,30.74,1022.0,96.0,1022.0,1018.0,-46.113180,-146.283944
558,558,2013921,Ust-Kuyga,99.0,197.0,RU,29.69,-37.35,-29.69,-29.69,1027.0,80.0,1027.0,1017.0,51.845297,-58.281622


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)
city_df = city_df.dropna()
locations = city_df[["Latitude", "Longitude"]].astype(float)

weights = city_df["Humidity"].astype(float)
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [4]:
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = city_df.loc[city_df['Humidity'] < 50]
ideal_weather = city_df.loc[city_df['Humidity'] > 30]
ideal_weather = city_df.loc[city_df['Feels Like'] < 74]
ideal_weather = city_df.loc[city_df['Feels Like'] > 68]
ideal_weather = city_df.loc[city_df['Max Temp'] < 84]
ideal_weather = city_df.loc[city_df['Min Temp'] > 58]
ideal_weather = city_df.loc[city_df['Wind Speed'] < 24]
ideal_weather = city_df.loc[city_df['Cloudiness'] > 40]
dropped_weather = ideal_weather.dropna(subset=['Humidity', 'Feels Like', 'Max Temp', 'Min Temp', 'Wind Speed', 'Cloudiness', 'Latitude', 'Longitude'])
hotel_checker = dropped_weather.sample(10)
hotel_checker

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Country,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
395,395,2122262,Palana,100.0,41.0,RU,1.58,-0.45,15.80,15.80,989.0,94.0,989.0,959.0,-13.732218,-44.404905
287,287,1634266,Negara,78.0,196.0,ID,83.95,89.80,83.95,83.95,1009.0,72.0,1009.0,1004.0,-9.243979,46.345690
498,498,2120048,Ust-Nera,87.0,31.0,RU,22.74,-32.28,-22.74,-22.74,1023.0,77.0,1023.0,957.0,76.077855,-6.171955
469,469,1511309,Asino,100.0,127.0,RU,7.73,-17.10,-7.73,-7.73,1039.0,89.0,1039.0,1025.0,85.474321,-83.146913
343,343,4033375,Tevaitoa,54.0,69.0,PF,81.43,80.04,81.43,81.43,1012.0,75.0,1012.0,1012.0,77.074425,110.292865
361,361,374739,Gogrial,69.0,23.0,SS,69.75,61.92,69.75,69.75,1010.0,22.0,1010.0,963.0,-38.407267,-67.167401
436,436,1492607,Severnoye,100.0,196.0,RU,6.42,-3.60,6.42,6.42,1035.0,95.0,1035.0,1019.0,26.638578,-7.670283
316,316,2018069,Peleduy,100.0,220.0,RU,6.44,-3.05,6.44,6.44,1016.0,92.0,1016.0,989.0,-51.728258,-174.957853
549,549,2025256,Chumikan,100.0,255.0,RU,9.81,-31.79,-9.81,-9.81,1012.0,79.0,1012.0,1012.0,74.612516,143.568358
39,39,2126123,Chokurdakh,100.0,347.0,RU,21.35,-30.32,-21.35,-21.35,1025.0,88.0,1025.0,1019.0,18.397916,-55.540313


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = hotel_checker.reindex(columns = ["City Name", "Country", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.reset_index().rename(columns={"City Name":"City", "Latitude":"Lat", "Longitude":"Lng"})
hotel_df

,index,City,Country,Lat,Lng,Hotel Name
0,395,Palana,RU,-13.732218,-44.404905,
1,287,Negara,ID,-9.243979,46.345690,
2,498,Ust-Nera,RU,76.077855,-6.171955,
3,469,Asino,RU,85.474321,-83.146913,
4,343,Tevaitoa,PF,77.074425,110.292865,
5,361,Gogrial,SS,-38.407267,-67.167401,
6,436,Severnoye,RU,26.638578,-7.670283,
7,316,Peleduy,RU,-51.728258,-174.957853,
8,549,Chumikan,RU,74.612516,143.568358,
9,39,Chokurdakh,RU,18.397916,-55.540313,


In [7]:
# find the closest restaurant of each type to coordinates


placeid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
coordinate_url = "https://maps.googleapis.com/maps/api/place/details/json?"
hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

hotels = []
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #get place_id for city name
    placeid_params = {
        "input": row['City'],
        "inputtype": "textquery",
        "key": gkey,
    }
    
    ID = requests.get(placeid_url, params=placeid_params).json()
    ID = (ID["candidates"][0]["place_id"])
    City = row['City']
    print(f'City = {City}')
    

#     use place_id to get coordinates that agree with google's api
    coordinate_params = {
        "place_id": ID,
        "key": gkey,
    }
#     assemble url and make API request
    coords_req = requests.get(coordinate_url, params=coordinate_params).json()
    lat = coords_req['result']['geometry']['location']['lat']
    lng = coords_req['result']['geometry']['location']['lng']
    try:
        Country = row["Country"]
        print(f'Country = {Country}')
    except:
        print(f'Country = n/a')

    
    coords = f'{lat},{lng}'
    
    hotel_params = {
        "location": coords,  
        "radius": 5000,
        "types": "lodging",
        "key": gkey,
    }
    
#     assemble url and make API request
    print(f"Retrieving Results for hotels near {City}:")
    response = requests.get(hotel_url, params=hotel_params).json()
    # extract results
    try:
        results = response['results'][0]['name']
        hotels.append(results)
        print(f'Hotel = {results}')
    except:
        results = "No hotel found nearby"
        hotels.append("n/a")
        print(results)
    
    
    if results == "No hotel found nearby":
        print(f"Sorry, there are no hotels within 5000 meters of {City}. Maybe try someplace less remote?")
    else:
        print(f"The closest hotel to {City} is {results}.")    
    print("------------")
hotel_df["Hotel Name"] = hotels
hotel_df.rename(columns={"Latitude":"Lat", "Longitude":"Lng"})
hotel_df


City = Palana
Country = RU
Retrieving Results for hotels near Palana:
No hotel found nearby
Sorry, there are no hotels within 5000 meters of Palana. Maybe try someplace less remote?
------------
City = Negara
Country = ID
Retrieving Results for hotels near Negara:
Hotel = Jati Hotel
The closest hotel to Negara is Jati Hotel.
------------
City = Ust-Nera
Country = RU
Retrieving Results for hotels near Ust-Nera:
Hotel = Gostevoy Dom
The closest hotel to Ust-Nera is Gostevoy Dom.
------------
City = Asino
Country = RU
Retrieving Results for hotels near Asino:
Hotel = Raduga
The closest hotel to Asino is Raduga.
------------
City = Tevaitoa
Country = PF
Retrieving Results for hotels near Tevaitoa:
Hotel = Hôtel Raiatea Lodge
The closest hotel to Tevaitoa is Hôtel Raiatea Lodge.
------------
City = Gogrial
Country = SS
Retrieving Results for hotels near Gogrial:
No hotel found nearby
Sorry, there are no hotels within 5000 meters of Gogrial. Maybe try someplace less remote?
------------
City

,index,City,Country,Lat,Lng,Hotel Name
0,395,Palana,RU,-13.732218,-44.404905,n/a
1,287,Negara,ID,-9.243979,46.345690,Jati Hotel
2,498,Ust-Nera,RU,76.077855,-6.171955,Gostevoy Dom
3,469,Asino,RU,85.474321,-83.146913,Raduga
4,343,Tevaitoa,PF,77.074425,110.292865,Hôtel Raiatea Lodge
5,361,Gogrial,SS,-38.407267,-67.167401,n/a
6,436,Severnoye,RU,26.638578,-7.670283,"Гостиница ""Охотник"""
7,316,Peleduy,RU,-51.728258,-174.957853,Гостиница «Уют»
8,549,Chumikan,RU,74.612516,143.568358,Fortuna
9,39,Chokurdakh,RU,18.397916,-55.540313,n/a


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:

# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…